# Object Detection

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import pandas as pd
import base64
import io
import math 
from math import sqrt
import argparse
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings
%matplotlib inline             

## Paths 

In [18]:
# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = 'C:\\Users\\DELL\\Downloads\\shortshirt.jpg'

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = 'E:\\Graduation Project\\IMPLEMENTATION\\tensorflow\\workspace\\training_demo\\exported-models\\my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = 'E:\\Graduation Project\\IMPLEMENTATION\\tensorflow\\workspace\\training_demo\\annotations\\label_map.pbtxt'

# Load The Model

In [19]:
# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "\\saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(IMAGE_PATHS), end='')


Loading model...Done! Took 14.415225505828857 seconds
Running inference for C:\Users\DELL\Downloads\shortshirt.jpg... 

## Visualize The Output

# Image Similarity

### Loading Pre-Trained Feature Extraction Model

### Converting Images To Feature Vectors (Embeddings)

### class to find the similarity between two feature vectors 

### Path Of Main Uploaded Image

### getting feature vector of the main uploaded image 

### looping through images in the dataset and comparing each image feature vector with the main image feature vector